In [144]:
import requests
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import Normalizer
import torch
import mglearn
from collections import Counter
from kiwipiepy import Kiwi, TypoTransformer, TypoDefinition
import re
import pandas as pd
import json
import numpy as np
from kiwipiepy.utils import Stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [146]:
normalizer = Normalizer()
clustered_dong = pd.read_json("./data/군집상권데이터.json")
menu_data = pd.read_json("./data/메뉴_추상설명_재료_토큰화.json")
cafe_menu_data = pd.read_json("./data/메뉴 데이터.json")

In [120]:
menu_data

,menu,info,ingred,tokenized_info
0,바닐라 라떼,따뜻 고소 편안 달콤 향긋 은은 포근 조화 균형감,"우유, 생크림, 설탕, 소금, 바닐라 추출물, 얼음",따뜻 고소 편안 달콤 향긋 은은 포근 조화 균형감
1,바질 에이드,신선 허브 향 상쾌 청량감 깨끗 독특 조화,"오렌지 주스, 라임 주스, 레몬 주스, 설탕, 물, 식용 색소, 젤라틴, 얼음",신선 허브 향 상쾌 청량감 깨끗 독특 조화
2,멜론 소다,시원하 달콤 과일 향 활기차 어리 시절 달콤쌉싸름 경쾌 기포,"크림치즈, 설탕, 계란, 사워크림, 바닐라 추출물, 레몬즙, 밀가루, 버터, 소금...",시원하 달콤 과일 향 활기차 어리 시절 달콤쌉싸름 경쾌 기포
3,크림 라떼,고급 풍미 따뜻 풍성 밀도 감 감 단맛 마무리,"아몬드 가루, 슈가파우더, 흰자, 설탕, 바닐라 추출물, 크림치즈, 쿠키 크럼블,...",고급 풍미 따뜻 풍성 밀도 감 감 단맛 마무리
4,유자 소다,상큼 청량감 과일 향 개운 여름 활력 감귤 향 맑 상쾌하 탄산 새콤하,"아몬드 가루, 슈가파우더, 흰자, 설탕, 바닐라 추출물, 체리 잼, 생크림, 버터...",상큼 청량감 과일 향 개운 여름 활력 감귤 향 맑 상쾌하 탄산 새콤하
...,...,...,...,...
1313,카라멜 머핀,달콤 고소 촉촉 질감 진하 캐러멜 풍미 클래식 디저트 고급 만족감 따뜻 풍미 깊 사랑,"박력분, 설탕, 베이킹파우더, 소금, 계란, 우유, 버터, 아몬드 슬라이스, 아몬...",달콤 고소 촉촉 질감 진하 캐러멜 풍미 클래식 디저트 고급 만족감 따뜻 풍미 깊 사랑
1314,아몬드 머핀,고소하 달콤 질감 촉촉 아몬드 풍미 건강 디저트 클래식 감성 만족감 화사 심플,"콜드 브루 커피, 우유, 꿀, 얼음, 바닐라 시럽, 초콜릿 시럽, 생크림",고소하 달콤 질감 촉촉 아몬드 풍미 건강 디저트 클래식 감성 만족감 화사 심플
1315,콜드 브루 딜라이트,진하 커피 풍미 크리미함 달콤 밸런스 고급 음료 클래식 감성 만족감 깔끔 농후 사랑,"고구마, 우유, 꿀, 얼음, 바닐라 아이스크림",진하 커피 풍미 크리미함 달콤 밸런스 고급 음료 클래식 감성 만족감 깔끔 농후 사랑
1316,아몬드 크럼블,고소하 바삭한크럼블토핑 달콤 촉촉 중심 건강 디저트 고급 만족감 화사 풍미 깊,"아몬드, 버터, 밀가루, 설탕, 소금, 계란, 베이킹파우더",고소하 바삭한크럼블토핑 달콤 촉촉 중심 건강 디저트 고급 만족감 화사 풍미 깊


In [ ]:
app = Flask(__name__)
CORS(app)  # 모든 도메인에서의 요청 허용

# CORS(app, resources={r"/api/*": {"origins": "http://localhost:8087"}})
# REST API key
KAKAO_API_KEY = "015412fbc48c3a4e31b1926b6adb667e"

# 1. 유사 상권 탐색
# 스프링 부트로부터 사용자가 클릭한 좌표 정보를 전달받아 동을 찾은 후 분석하고 전송
@app.route('/send-coordinates', methods=['POST'])
def receive_coordinates():
    # 클라이언트로부터 받은 데이터(현 위치 위도, 경도)
    data = request.get_json()
    latitude = data.get('latitude')
    longitude = data.get('longitude')
    
    url = f"https://dapi.kakao.com/v2/local/geo/coord2address.json?x={longitude}&y={latitude}"
    headers = {"Authorization" : f"KakaoAK {KAKAO_API_KEY}"}
    
    response = requests.get(url, headers=headers)
    
    
    dong_number = get_dong_number(latitude, longitude)
    if(dong_number == None):
        return jsonify({"status" : "error", "message" : "현재 지역의 법정동 번호를 찾을 수 없습니다."})
    
    # 유사 상권의 법정동 번호
    similar_dongs = yusa_dong(clustered_dong, dong_number)
    
    # 스트링 부트로 전달
    spring_response = similar_dong_send_to_spring(similar_dongs)
    # 전송 결과 확인
    if(spring_response.get("status")=="success"):
        return jsonify({"status":"success",
                        "yusadongs":similar_dongs,
                        "message": "flask successed"
                       })
    else:
        return jsonify({
            "status" : "error",
            "yusadongs" : "",
            "message" : spring_response.get("error")
        })



# 스프링 부트로부터 키워드 전달받아 분석
@app.route("/analyze-keyword", methods=["POST"])
def analyze_keyword():
    # 요청 본문 데이터 확인 : 키워드
    data = request.get_json()
    keyword = data.get("keyword", "")
    recommanded = input_simil(menu_data, keyword)

    spring_response = keyword_menu_send_to_spring(keyword, recommanded)
    if (spring_response.get("status") == "success"):
        return jsonify({"status": "success",
                        "keyword": keyword,
                        "recommanded": recommanded
                        })
    else:
        return jsonify({
            "status": "error",
            "keyword": keyword,
            "message": spring_response.get("error")
        })

# 스프링 부트로부터 메뉴 전달받아 분석
@app.route("/analyze-menu", methods=["POST"])
def analyze_menu():
    # 요청 데이터 확인 : 메뉴
    data = request.get_json()
    menu_name = data.get("menu","")
    print(menu_name)
    similar_menus = ingred_sim(menu_data, menu_name)
    
    spring_response = similar_menu_recommanded(similar_menus, menu_name)
#     print(spring_response["similarMenus"])
    if(spring_response.get("status")=="success"):
        return jsonify({"status":"success",
                        "menu":menu_name,
                       "similarMenus":similar_menus})
    else:
        return jsonify({"status":"error",
                       "message":spring_response.get("error")})

@app.route("/sanggueon-gongtong", methods=["POST"])
def sanggueon_gongtong():
    print("!!!")
    data = request.get_json()
    sang1 = data.get("sang1", "")
    sang2 = data.get("sang2", "")
    print(sang1)
    gongtong_menus = yusasang_gongtong(cafe_menu_data, sang1, sang2)
    print(gongtong_menus)
    spring_response = search_sanggueon_gongtong(gongtong_menus,sang1, sang2)
    if(spring_response.get("status")=="success"):
        return jsonify({"status":"success",
                       "gongtongMenus":gongtong_menus})
    else:
        return jsonify({"status":"error",
                       "message":spring_response.get("error")})
    


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.219.227:5001
Press CTRL+C to quit
192.168.219.227 - - [03/Jan/2025 11:01:38] "POST /analyze-menu HTTP/1.1" 200 -


얼그레이
메뉴 이름이 잘못되었습니다.
{'status': 'success', 'similarMenus': ['유사한 메뉴가 존재하지 않습니다...'], 'menu': '얼그레이'}


192.168.219.227 - - [03/Jan/2025 11:01:50] "POST /analyze-menu HTTP/1.1" 200 -


얼그레이 티
[1.         0.27405115 0.26946069 0.62680058 0.0642052  0.32587373
 0.36705578 0.23136077 0.169454   0.48319478 0.21969895 0.18471705
 0.24102714 0.26133952 0.11186964 0.12762822 0.16147758 0.114822
 0.15091044 0.37794463 0.54896543 0.16301969 0.08202175 0.09151225
 0.16543099 0.05354461 0.23116512 0.25033291 0.22583968 0.28280885
 0.2569506  0.0628027  0.17268263 0.17083422 0.02839503 0.13871033
 0.13320079 0.0764668  0.11193355 0.02668866 0.05583608 0.16259532
 0.         0.13046379 0.         0.05752985 0.32605623 0.20462601
 0.13131673 0.10340954]
{'status': 'success', 'similarMenus': ['얼그레이 스무디', '홍차 스무디', '얼그레이 버블티', '디카페인 카라멜 마끼야또', '얼그레이 아메리카노', '샤인머스켓 스무디', '라벤더 밀크티', '홍차 크림 아인슈페너', '얼그레이 밀크티', '자스민 티'], 'menu': '얼그레이 티'}


...

캐모마일 티
[1.         0.36710496 0.40141349 0.31137665 0.43373751 0.19055949
 0.21362    0.20552091 0.17057744 0.09387071 0.12623146 0.34721255
 0.27104074 0.18767799 0.03485927 0.17454369 0.20334284 0.13271213
 0.08318483 0.24276689 0.14819761 0.35331427 0.07230801 0.19081055
 0.11414182 0.14103093 0.18179861 0.06295031 0.11743115 0.17852426
 0.25447907 0.11202132 0.11209006 0.06321883 0.06419814 0.22430038
 0.07509341 0.26387111 0.20029151 0.06568088 0.13201602 0.17873728
 0.16089092 0.         0.12530458 0.13738799 0.15139937 0.170239
 0.15606477 0.06133173]
{'status': 'success', 'similarMenus': ['자스민 티', '아카시아 티', '카모마일 티', '매실차', '오미자차', '민트 티', '히비스커스 아이스티', '청귤차', '라즈베리 티', '얼그레이 스무디'], 'menu': '캐모마일 티'}


192.168.219.227 - - [03/Jan/2025 11:46:06] "POST /analyze-menu HTTP/1.1" 200 -


바닐라 라떼
[1.         0.0787072  0.26393837 0.25862481 0.12309721 0.17156755
 0.05748665 0.10370115 0.09795989 0.11582983 0.32435298 0.63157982
 0.18703241 0.19912193 0.13985302 0.387336   0.11375889 0.11672947
 0.26566882 0.08027804 0.29207931 0.27537869 0.06498383 0.09485563
 0.09312026 0.21914318 0.13574792 0.09686027 0.05665338 0.28594818
 0.14574599 0.26466601 0.32079248 0.14974248 0.22354459 0.1489078
 0.04738369 0.35222841 0.09315342 0.05470673 0.02078002 0.10146214
 0.15460419 0.09946421 0.22898407 0.14616101 0.19003191 0.27621835
 0.11319365 0.11609711]
{'status': 'success', 'similarMenus': ['아몬드 라떼', '헤이즐넛 아메리카노', '헤이즐넛 콜드브루', '고구마 라떼', '진저 라떼', '고구마 밀크티', '카라멜 카푸치노', '달고나 밀크티', '오트밀 라떼', '카페라떼'], 'menu': '바닐라 라떼'}


192.168.219.227 - - [03/Jan/2025 12:48:51] "POST /analyze-menu HTTP/1.1" 200 -


꿀 아메리카노
[1.         0.32603965 0.20099782 0.40968764 0.2770676  0.33751453
 0.32441021 0.26113316 0.1836642  0.34062506 0.3223986  0.35976541
 0.20528844 0.44520155 0.21479441 0.31140666 0.22234092 0.22889802
 0.27808028 0.19731663 0.22498437 0.27482318 0.23798913 0.25445164
 0.23459658 0.15970013 0.13587952 0.26133166 0.24172784 0.41247759
 0.44318992 0.03754769 0.17746286 0.54041034 0.21187331 0.25088046
 0.23768599 0.27092087 0.23615129 0.19962492 0.20798894 0.33182171
 0.39224228 0.16215599 0.26212796 0.16999049 0.10706514 0.21978669
 0.28868069 0.0903906 ]
{'status': 'success', 'similarMenus': ['바닐라 아인슈페너', '홍차 스무디', '에스프레소 라떼', '헤이즐넛 에스프레소', '돌체 에스프레소', '콜드브루 모카', '콜드브루 라떼', '바닐라 프라푸치노', '콜드브루 카푸치노', '크림 아인슈페너'], 'menu': '꿀 아메리카노'}


In [148]:
def data_tokenizing(data):
    data["ingred"] = data["ingred"].replace('\'', '', regex=True)
    kiwi = Kiwi()
    for i in range(len(data["info"])):
        data.iloc[i, 1] = kiwi_tokenizer(data["info"][i], tokenizer=kiwi)
    return data

In [150]:

def ingred_sim(data, menu_name, num=10):
    # info_simil 함수로 유사한 메뉴 추출
    info_sim = info_simil(data, menu_name, 50)
    
    # 유사한 메뉴가 없는 경우 처리
    if info_sim.empty or len(info_sim["menu"]) == 0:
        return ["유사한 메뉴가 존재하지 않습니다..."]

    # TF-IDF 변환
    tfidfv = TfidfVectorizer().fit_transform(info_sim["tokenized_info"])

    # 코사인 유사도 계산
    ingred_simil = cosine_similarity(tfidfv[0], tfidfv)[0]
    print(ingred_simil)
    # 자기 자신 제외하고 유사도 순으로 정렬
    ingred_simil_index = np.argsort(ingred_simil)[::-1][1:num + 1]

    # 유사도가 낮은 경우 처리
    if len(ingred_simil_index) == 0 or ingred_simil[ingred_simil_index[0]] < 0.2:
        return ["유사한 메뉴가 존재하지 않습니다..."]

    # 결과 반환
    return list(info_sim.iloc[ingred_simil_index]["menu"])

In [152]:
# 메뉴의 설명이 유사한 메뉴 추천
def info_simil(data, menu_name, num=10):
    # 메뉴 이름이 유효한지 확인
    if menu_name not in data["menu"].values:
        print("메뉴 이름이 잘못되었습니다.")
        return pd.DataFrame()  # 빈 데이터프레임 반환

    # 선택된 메뉴의 인덱스 가져오기
    pick_idx = data[data["menu"] == menu_name].index[0]

    # TF-IDF 변환
    tfidfv = TfidfVectorizer().fit_transform(data["tokenized_info"])

    # 코사인 유사도 계산
    cosine_simil = cosine_similarity(tfidfv[pick_idx], tfidfv)[0]

    # 유사도를 기준으로 정렬 (내림차순)
    simil_index = np.argsort(cosine_simil)[::-1]

    # 자기 자신 제외하고 상위 num개 선택
    simil_index = simil_index[1:num + 1]

    # 유사도가 일정 수준 이하인 경우 처리
    if len(simil_index) == 0 or cosine_simil[simil_index[0]] < 0.2:
        print("유사한 메뉴가 존재하지 않습니다.")
        return pd.DataFrame()  # 빈 데이터프레임 반환

    # 결과 반환
    return data.iloc[simil_index]

In [17]:
info_simil(menu_data,"카페모카")

,menu,info,ingred,tokenized_info
201,모카 카푸치노,초콜릿 진하 맛 크리미함 달콤 풍미 진하 초콜릿 향 크림 깊 풍부 맛 따뜻 여운,"밀가루, 설탕, 계란, 우유, 버터, 이스트, 밤, 크림, 바닐라 추출물",초콜릿 진하 맛 크리미함 달콤 풍미 진하 초콜릿 향 크림 깊 풍부 맛 따뜻 여운
85,초콜렛 모카,진하 초콜릿 풍부 달콤 따뜻 만족감 달콤하 초콜릿 진하 크림 고급 풍미 풍부 따뜻 마무리,"말차 가루, 우유, 설탕, 얼음, 휘핑크림",진하 초콜릿 풍부 달콤 따뜻 만족감 달콤하 초콜릿 진하 크림 고급 풍미 풍부 따뜻 마무리
77,모카 라떼,달콤 초콜릿 진하 커피 향 만족감 진하 초콜릿 풍부하 크림 감 고급 달콤 끝 맛 따...,"홍차, 우유, 설탕, 바닐라 추출물, 얼음",달콤 초콜릿 진하 커피 향 만족감 진하 초콜릿 풍부하 크림 감 고급 달콤 끝 맛 따...
253,크림 카페모카,초콜릿 크리미함 달콤 진하 맛 고급 진하 초콜릿 풍미 크림 달콤 조화 깊 풍부 맛 ...,"밀가루, 설탕, 버터, 우유, 라즈베리, 베이킹파우더, 계란",초콜릿 크리미함 달콤 진하 맛 고급 진하 초콜릿 풍미 크림 달콤 조화 깊 풍부 맛 ...
418,밀크 쇼콜라,초콜릿 달콤 풍미 따뜻 진하 초콜릿 향 달콤 감미 만족감 고급 맛,"복숭아, 플레인 요거트, 꿀, 우유, 얼음 (옵션), 민트 잎 (옵션)",초콜릿 달콤 풍미 따뜻 진하 초콜릿 향 달콤 감미 만족감 고급 맛
325,초코 프라푸치노,진하 초콜릿 달콤 풍미 만족감 진하 초콜릿 향 조화 달콤 만족감 고급 풍미,"밀가루, 밤, 계란, 설탕, 버터, 베이킹파우더, 소금, 바닐라 엑스트랙",진하 초콜릿 달콤 풍미 만족감 진하 초콜릿 향 조화 달콤 만족감 고급 풍미
208,초코 스무디,진하 초콜릿 달콤 크리미함 만족감 풍부 진하 초콜릿 풍미 만족감 세련 조화,"타르트 반죽, 자두, 설탕, 크림치즈, 생크림, 바닐라 엑스트랙트",진하 초콜릿 달콤 크리미함 만족감 풍부 진하 초콜릿 풍미 만족감 세련 조화
497,초코 마들렌,달콤 진하 초콜릿 촉촉 감성 클래식 디저트 풍미 깊 만족감 고급,"샤인머스켓, 우유, 바닐라 아이스크림, 설탕, 얼음",달콤 진하 초콜릿 촉촉 감성 클래식 디저트 풍미 깊 만족감 고급
294,모카 스무디,진하 초콜릿 달콤 풍미 만족감 진하 초콜릿 풍미 조화 달콤 여운 고급 맛,"에스프레소, 우유, 휘핑크림, 설탕, 바닐라 시럽",진하 초콜릿 달콤 풍미 만족감 진하 초콜릿 풍미 조화 달콤 여운 고급 맛
108,쇼콜라 쉐이크,진하 초콜릿 달콤 크리미함 만족감 풍미 풍부하 크림 감 달콤 만족감 디저트 감성,"우롱차, 우유, 설탕, 얼음, 바닐라 시럽",진하 초콜릿 달콤 크리미함 만족감 풍미 풍부하 크림 감 달콤 만족감 디저트 감성


In [154]:
def input_simil(data, input_data, num=10):
    input_token = kiwi_tokenizer(input_data)
    text_vector = np.array(data["menu"] + " " + data["tokenized_info"])
    t_vector = np.insert(text_vector, 0, input_token)

    tfidfv = TfidfVectorizer().fit_transform(t_vector)
    cosine_simil = cosine_similarity(tfidfv[0], tfidfv)[0]

    simil_index = np.flip(np.argsort(cosine_simil[1:]))  # Exclude self-similarity

    # Lower threshold to capture more matches
    threshold = 0.15
    if cosine_simil[simil_index[0] + 1] < threshold:
        return ["유사한 메뉴가 존재하지 않습니다..."]
    else:
        top_matches = data.iloc[simil_index[:num], 0]
        return list(top_matches)

In [142]:
recommanded2 = ingred_sim(menu_data, "얼그레이 티")
recommanded2

[1.         0.27405115 0.26946069 0.62680058 0.0642052  0.32587373
 0.36705578 0.23136077 0.169454   0.48319478 0.21969895 0.18471705
 0.24102714 0.26133952 0.11186964 0.12762822 0.16147758 0.114822
 0.15091044 0.37794463 0.54896543 0.16301969 0.08202175 0.09151225
 0.16543099 0.05354461 0.23116512 0.25033291 0.22583968 0.28280885
 0.2569506  0.0628027  0.17268263 0.17083422 0.02839503 0.13871033
 0.13320079 0.0764668  0.11193355 0.02668866 0.05583608 0.16259532
 0.         0.13046379 0.         0.05752985 0.32605623 0.20462601
 0.13131673 0.10340954]


['얼그레이 스무디',
 '홍차 스무디',
 '얼그레이 버블티',
 '디카페인 카라멜 마끼야또',
 '얼그레이 아메리카노',
 '샤인머스켓 스무디',
 '라벤더 밀크티',
 '홍차 크림 아인슈페너',
 '얼그레이 밀크티',
 '자스민 티']

In [156]:
def kiwi_tokenizer(sentence, tokenizer=Kiwi(typos="basic")):
    sentence = sentence.replace(" ", "")
    stopwords = Stopwords()
    get_tags = ["XR", "NNG", "VA"]
    sentence = tokenizer.space(sentence)

    kiwi_result = tokenizer.tokenize(
        sentence, normalize_coda=True, stopwords=stopwords)
    token_list = []
    for token in range(len(kiwi_result)):
        if (kiwi_result[token].tag in get_tags):
            token_list.append(kiwi_result[token].form)
    token_str = " ".join(token_list)
    return token_str

In [158]:
def yusa_dong(clustered_dong, dong_pk):
    target_dong_index = clustered_dong[clustered_dong["법정동 번호(PK)"]==dong_pk].index[0]

    same_cluster_dong = clustered_dong[clustered_dong["군집"]==clustered_dong.iloc[target_dong_index,-1]]
    same_cluster_dong.reset_index(drop=True, inplace=True)
    target_dong_index = same_cluster_dong[same_cluster_dong["법정동 번호(PK)"]==dong_pk].index[0]
    
    simil_data = same_cluster_dong.iloc[:,3:-1]
    simil_data_scaled = normalizer.fit_transform(simil_data)

    train_tensor = torch.tensor(simil_data_scaled, dtype=torch.float64)
    euclidean_distances = torch.cdist(train_tensor[target_dong_index].unsqueeze(0), train_tensor).squeeze()
    top_idx = np.argsort(euclidean_distances)[1:10]

    return [int(same_cluster_dong["법정동 번호(PK)"].iloc[int(idx)]) for idx in top_idx]

In [160]:
# 키워드 추천 분석 결과 전달
def keyword_menu_send_to_spring(keyword, recommanded):
    spring_url = "http://localhost:8087/api/receive-keyword-menu"
    payload = {
        "status" : "success",
        "keyword": keyword,
        "recommanded": recommanded
    }
    # json 형식으로 전송
    headers = {"Content-Type": "application/json"}
    print(payload)
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()
        return {"status": "success", "message": "스프링으로 전송 완료"}
    except Exception as e:
        # 예외처리
        print(f"스프링 부트 전송 실패: {str(e)}")
        return {"status": "error", "error": str(e)}

In [162]:
# 분석한 유사 상권 정보 전달
def similar_dong_send_to_spring(similar_dongs):
    spring_url = "http://localhost:8087/api/receive-similar-dongs"
    # 스프링에 dong이라는 이름으로 dong_name 데이터 전송
    payload = {"similarDongs": similar_dongs,
               "status": "success"
              }
    # json 형식으로 전송
    headers = {"Content-Type": "application/json"}
    
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()  # HTTP 오류 있는지 확인 => 예외 발생
        return {"status": "success", "message": "스프링으로 전송 완료"}
    except Exception as e:
        # 예외처리
        print(f"스프링 전송 실패: {str(e)}")
        return {"status": "error", "error": str(e)}

In [164]:
# 유사 메뉴 분석 결과 스프링으로 전송
def similar_menu_recommanded(similar_menus, menu_name):
    spring_url="http://localhost:8087/api/receive-similar-menus"
    
    payload={"status":"success",
            "similarMenus":similar_menus,
            "menu": menu_name
            }
        
    headers={"Content-Type" : "application/json"}
    print(payload)
    
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()
        return {"status":"success","message":"스프링으로 전송 완료"}
    except Exception as e:
        print(f"스프링 전송 실패: {str(e)}")
        return {"status" : "error", "error":str(e)}

In [166]:
# 상권 공통 메뉴 분석 결과 스프링으로 전송
def search_sanggueon_gongtong(gongtong_menus, sang1, sang2):
    spring_url="http://localhost:8087/api/receive-sanggueon-gongtong"
    
    payload={"status":"success",
            "gongtongMenus" : gongtong_menus,
             "sang1" : sang1,
             "sang2":sang2
            }
        
    headers={"Content-Type" : "application/json"}
    print(payload)
    
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()
        return {"status":"success","message":"스프링으로 전송 완료"}
    except Exception as e:
        print(f"스프링 전송 실패: {str(e)}")
        return {"status" : "error", "error":str(e)}

In [168]:
# 좌표를 통해 법정동 번호 조회
def get_dong_number(latitude, longitude, dong_data=clustered_dong):
    url = f"https://dapi.kakao.com/v2/local/geo/coord2address.json?x={longitude}&y={latitude}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}

    response = requests.get(url, headers=headers)

    # 200번 : 통신 성공
    if response.status_code == 200:
        address_info = response.json()
        # 카카오 api에 해당 좌표 정보가 있는지 확인
        if address_info['documents']:
            addr = address_info['documents'][0]['address']['address_name']
            region = addr.split(" ")[0]
            gu = addr.split(" ")[1]
            dong = addr.split(" ")[2]

            region_dong = dong_data[dong_data["지역"].str.contains(region)]
            dong_number = list(
                region_dong[region_dong["동 이름"] == dong].head(1)["법정동 번호(PK)"])[0]
            return dong_number
    return None

In [170]:
# 두 상권 중 다른 상권에만 유독 더 많이 파는 메뉴
def yusasang_only(data, sang1, sang2):
    sang1data = data[data["dong"]==sang1]
    sang2data = data[data["dong"]==sang2]
    
    sang1_count = Counter(sang1data['menu_keyword'])
    sang2_count = Counter(sang2data['menu_keyword'])

    sang1_menu = list(sang1_count.keys())
    sang1_cnt = sang1_count.values()
    sang1_cnt_list = list(sang1_cnt)
    sang1_rat = []
    sang1_sum = 0

    sang2_menu = list(sang2_count.keys())
    sang2_cnt = sang2_count.values()
    sang2_cnt_list = list(sang2_cnt)
    sang2_rat = []
    sang2_sum = 0

    for i in range(len(sang1_cnt)):
        sang1_sum+=sang1_cnt_list[i]
    for i in range(len(sang2_cnt)):
        sang2_sum+=sang2_cnt_list[i]

    sang1_rat = np.array([r/sang1_sum for r in sang1_cnt_list])
    sang2_rat = np.array([r/sang2_sum for r in sang2_cnt_list])

    sang1_most_rat = np.flip(np.argpartition(sang1_rat, kth=-1)[:])
    sang2_most_rat = np.flip(np.argpartition(sang2_rat, kth=-1)[:])

    sang1_most = [sang1_menu[menu_i] for menu_i in sang1_most_rat]
    sang2_most = [sang2_menu[menu_i] for menu_i in sang2_most_rat]

    only_sang2 = []
    popularity = len(sang1_most)//5
    
    for i in range(len(sang2_most)):
        if(sang2_most[i] not in sang1_most[:popularity]):
            only_sang2.append(sang2_most[i])
        if(len(only_sang2) == 5):
            break
    if(len(only_sang2) < 3):
        only_sang2.append("두 상권의 거의 모든 메뉴가 유사합니다.")

    return only_sang2

In [172]:
#두 상권 모두 많이 파는 메뉴
def yusasang_gongtong(data, sang1, sang2):
    sang1data = data[data["dong"]==sang1]
    sang2data = data[data["dong"]==sang2]
    
    sang1_count = Counter(sang1data['menu_keyword'])
    sang2_count = Counter(sang2data['menu_keyword'])

    sang1_menu = list(sang1_count.keys())
    sang1_cnt = sang1_count.values()
    sang1_cnt_list = list(sang1_cnt)
    sang1_rat = []
    sang1_sum = 0

    sang2_menu = list(sang2_count.keys())
    sang2_cnt = sang2_count.values()
    sang2_cnt_list = list(sang2_cnt)
    sang2_rat = []
    sang2_sum = 0

    for i in range(len(sang1_cnt)):
        sang1_sum+=sang1_cnt_list[i]
    for i in range(len(sang2_cnt)):
        sang2_sum+=sang2_cnt_list[i]

    sang1_rat = np.array([r/sang1_sum for r in sang1_cnt_list])
    sang2_rat = np.array([r/sang2_sum for r in sang2_cnt_list])

    sang1_most_rat = np.flip(np.argpartition(sang1_rat, kth=-1)[:])
    sang2_most_rat = np.flip(np.argpartition(sang2_rat, kth=-1)[:])

    sang1_most = [sang1_menu[menu_i] for menu_i in sang1_most_rat]
    sang2_most = [sang2_menu[menu_i] for menu_i in sang2_most_rat]
    
    popularity = len(sang2_most) // 10
    
    gongtong_sang = []
    for i in range(popularity, len(sang1_most)):
        if(sang1_most[i] in sang2_most[:popularity]):
            gongtong_sang.append(sang1_most[i])
        if(len(gongtong_sang)==5):
            break
    if(len(gongtong_sang) < 3):
        gongtong_sang.append("두 상권의 메뉴는 거의 유사하지 않습니다.")
    return gongtong_sang

In [174]:
# 현재 상권에서 주로 파는 메뉴
def sang_menu(data, sang, no_menus=[]):
    sangdata = data[data["dong"]==sang]
    
    sang_count = Counter(sangdata['menu_keyword'])
    sang_most = sang_count.most_common(len(sang_count))
    
    sang_best_menu = []
    for i in range(5, len(sang_most)):
        if(sang_most[i][0] in no_menus):
            continue
        sang_best_menu.append(sang_most[i][0])
        if(len(sang_best_menu)==5):
            break
    if(len(sang_best_menu)<3):
        sang_best_menu.append("해당 상권은 분석할 수 없습니다. 사유 : 상권 내 카페 수 부족")
    return sang_best_menu

In [98]:
import sys

In [99]:
sys.version

'3.11.9 | packaged by Anaconda, Inc. | (main, Apr 19 2024, 16:40:41) [MSC v.1916 64 bit (AMD64)]'

In [100]:
from kiwipiepy import Kiwi

# Kiwi 객체 생성
kiwi = Kiwi()

# 텍스트 입력
input_text = "안녕하세요! 파이썬을 배우고 있습니다."

# 토큰화
tokenized = kiwi.tokenize(input_text)

# 결과 출력
for word in tokenized:
    print(word.form)

안녕
하
세요
!
파이썬
을
배우
고
있
습니다
.
